In [1]:
import os
import json
import pickle
import re
from collections import defaultdict, Counter
from typing import List, Dict, Set, Tuple
import fnmatch

class WildcardSearchSystem:
    def __init__(self, corpus_folder: str):
        """
        Initialize Wildcard Search System
        
        Args:
            corpus_folder: Path to the cleaned_corpus folder
        """
        self.corpus_folder = corpus_folder
        self.index_folder = os.path.join(corpus_folder, "wildcard_index")
        
        # Create index folder if it doesn't exist
        if not os.path.exists(self.index_folder):
            os.makedirs(self.index_folder)
        
        # Data structures
        self.documents = {}  # doc_id -> document info
        self.term_index = defaultdict(set)  # term -> set of doc_ids
        self.kgram_index = defaultdict(set)  # k-gram -> set of terms
        self.term_docs = defaultdict(set)  # term -> set of doc_ids
        self.doc_terms = defaultdict(set)  # doc_id -> set of terms
        
        # Statistics
        self.stats = {
            'total_documents': 0,
            'total_terms': 0,
            'unique_terms': 0,
            'kgram_size': 3  # For k-gram indexing
        }
    
    def build_kgram_index(self, term: str, k: int = 3) -> Set[str]:
        """
        Generate k-grams for a term
        
        Args:
            term: The term to generate k-grams for
            k: Size of k-grams (default 3)
        
        Returns:
            Set of k-grams
        """
        kgrams = set()
        # Add special characters to handle wildcards properly
        padded_term = f"${term}$"
        
        for i in range(len(padded_term) - k + 1):
            kgram = padded_term[i:i+k]
            kgrams.add(kgram)
        
        return kgrams
    
    def build_index(self):
        """Build wildcard search index from corpus"""
        print("🔨 Building Wildcard Search Index...")
        
        # Load document tokens
        doc_tokens_file = os.path.join(self.corpus_folder, "document_tokens.json")
        
        if not os.path.exists(doc_tokens_file):
            print("❌ document_tokens.json not found!")
            return False
        
        try:
            with open(doc_tokens_file, 'r', encoding='utf-8') as f:
                doc_data = json.load(f)
            print(f"✅ Loaded {len(doc_data)} documents")
        except Exception as e:
            print(f"❌ Error loading document tokens: {e}")
            return False
        
        # Process each document
        doc_id = 0
        all_terms = set()
        
        for doc_name, doc_info in doc_data.items():
            doc_id += 1
            tokens = doc_info.get('tokens', [])
            token_count = doc_info.get('token_count', 0)
            
            if tokens and token_count > 0:
                doc_key = f"doc_{doc_id:05d}"
                
                # Store document info
                self.documents[doc_key] = {
                    'name': doc_name,
                    'token_count': token_count
                }
                
                # Process unique terms in document
                unique_terms = set(tokens)
                self.doc_terms[doc_key] = unique_terms
                all_terms.update(unique_terms)
                
                # Add to term index
                for term in unique_terms:
                    self.term_docs[term].add(doc_key)
        
        # Build k-gram index for all terms
        print("📊 Building k-gram index for wildcard support...")
        k = self.stats['kgram_size']
        
        for term in all_terms:
            kgrams = self.build_kgram_index(term, k)
            for kgram in kgrams:
                self.kgram_index[kgram].add(term)
        
        # Update statistics
        self.stats['total_documents'] = len(self.documents)
        self.stats['unique_terms'] = len(all_terms)
        self.stats['total_terms'] = sum(len(terms) for terms in self.doc_terms.values())
        
        # Save index
        self.save_index()
        
        print(f"\n✅ Wildcard index built successfully!")
        print(f"   Documents: {self.stats['total_documents']:,}")
        print(f"   Unique terms: {self.stats['unique_terms']:,}")
        print(f"   Total term occurrences: {self.stats['total_terms']:,}")
        print(f"   K-gram size: {k}")
        print(f"   Unique k-grams: {len(self.kgram_index):,}")
        
        # Show sample terms and k-grams
        sample_terms = list(all_terms)[:10]
        print(f"   Sample terms: {', '.join(sample_terms)}")
        
        return True
    
    def save_index(self):
        """Save index to disk"""
        index_data = {
            'documents': self.documents,
            'kgram_index': {k: list(v) for k, v in self.kgram_index.items()},
            'term_docs': {k: list(v) for k, v in self.term_docs.items()},
            'doc_terms': {k: list(v) for k, v in self.doc_terms.items()},
            'stats': self.stats
        }
        
        index_file = os.path.join(self.index_folder, "wildcard_index.pkl")
        with open(index_file, 'wb') as f:
            pickle.dump(index_data, f)
        
        print(f"\n💾 Index saved to: {index_file}")
    
    def load_index(self):
        """Load index from disk"""
        index_file = os.path.join(self.index_folder, "wildcard_index.pkl")
        
        if not os.path.exists(index_file):
            print("❌ Index not found. Building index...")
            return self.build_index()
        
        try:
            print(f"📂 Loading index from: {index_file}")
            with open(index_file, 'rb') as f:
                index_data = pickle.load(f)
            
            self.documents = index_data['documents']
            self.stats = index_data['stats']
            
            # Reconstruct kgram_index
            self.kgram_index = defaultdict(set)
            for kgram, terms in index_data['kgram_index'].items():
                self.kgram_index[kgram] = set(terms)
            
            # Reconstruct term_docs
            self.term_docs = defaultdict(set)
            for term, docs in index_data['term_docs'].items():
                self.term_docs[term] = set(docs)
            
            # Reconstruct doc_terms
            self.doc_terms = defaultdict(set)
            for doc_id, terms in index_data['doc_terms'].items():
                self.doc_terms[doc_id] = set(terms)
            
            print(f"✅ Wildcard index loaded successfully!")
            print(f"   Documents: {self.stats['total_documents']:,}")
            print(f"   Unique terms: {self.stats['unique_terms']:,}")
            print(f"   K-gram size: {self.stats['kgram_size']}")
            
            return True
        except Exception as e:
            print(f"❌ Error loading index: {e}")
            return self.build_index()
    
    def parse_wildcard_pattern(self, pattern: str) -> Tuple[str, str]:
        """
        Parse wildcard pattern to extract prefix and suffix
        
        Args:
            pattern: Wildcard pattern (e.g., "court*", "*evidence", "m*rder")
        
        Returns:
            (pattern_type, regex_pattern)
        """
        pattern = pattern.lower().strip()
        
        if not pattern:
            return "empty", ""
        
        # Convert wildcard pattern to regex
        # Escape regex special characters except * and ?
        regex_pattern = re.escape(pattern)
        
        # Convert * to .*
        regex_pattern = regex_pattern.replace(r'\*', '.*')
        
        # Convert ? to .
        regex_pattern = regex_pattern.replace(r'\?', '.')
        
        # Add word boundaries for exact matching
        regex_pattern = f"^{regex_pattern}$"
        
        return "regex", regex_pattern
    
    def kgram_wildcard_search(self, pattern: str) -> Set[str]:
        """
        Wildcard search using k-gram indexing (fast)
        
        Args:
            pattern: Wildcard pattern
        
        Returns:
            Set of matching terms
        """
        pattern = pattern.lower().strip()
        
        if '*' not in pattern and '?' not in pattern:
            # No wildcards, exact match
            if pattern in self.term_docs:
                return {pattern}
            return set()
        
        # Generate k-grams from pattern
        k = self.stats['kgram_size']
        
        # Remove wildcards for k-gram generation
        clean_pattern = pattern.replace('*', '').replace('?', '')
        if not clean_pattern:
            # Pattern is only wildcards, return all terms
            return set(self.term_docs.keys())
        
        # Generate k-grams from clean pattern
        pattern_kgrams = self.build_kgram_index(clean_pattern, k)
        
        # Find candidate terms that share k-grams with pattern
        candidate_terms = None
        
        for kgram in pattern_kgrams:
            terms_with_kgram = self.kgram_index.get(kgram, set())
            
            if candidate_terms is None:
                candidate_terms = terms_with_kgram.copy()
            else:
                candidate_terms = candidate_terms.intersection(terms_with_kgram)
            
            if not candidate_terms:
                break
        
        if not candidate_terms:
            return set()
        
        # Filter candidate terms using regex
        _, regex_pattern = self.parse_wildcard_pattern(pattern)
        
        matching_terms = set()
        for term in candidate_terms:
            if re.match(regex_pattern, term):
                matching_terms.add(term)
        
        return matching_terms
    
    def regex_wildcard_search(self, pattern: str) -> Set[str]:
        """
        Wildcard search using regex (more accurate but slower)
        
        Args:
            pattern: Wildcard pattern
        
        Returns:
            Set of matching terms
        """
        pattern = pattern.lower().strip()
        
        if '*' not in pattern and '?' not in pattern:
            # No wildcards, exact match
            if pattern in self.term_docs:
                return {pattern}
            return set()
        
        # Convert to regex
        _, regex_pattern = self.parse_wildcard_pattern(pattern)
        
        # Search through all terms
        matching_terms = set()
        
        for term in self.term_docs.keys():
            if re.match(regex_pattern, term):
                matching_terms.add(term)
        
        return matching_terms
    
    def wildcard_search(self, pattern: str, use_kgram: bool = True) -> List[Dict]:
        """
        Wildcard search for documents
        
        Args:
            pattern: Wildcard pattern
            use_kgram: Use k-gram indexing (faster) or regex (more accurate)
        
        Returns:
            List of matching documents
        """
        if not pattern or not pattern.strip():
            print("❌ Empty pattern")
            return []
        
        print(f"\n🔍 Wildcard search: '{pattern}'")
        print(f"   Method: {'k-gram indexing' if use_kgram else 'regex scanning'}")
        
        # Find matching terms
        start_time = time.time()
        
        if use_kgram:
            matching_terms = self.kgram_wildcard_search(pattern)
        else:
            matching_terms = self.regex_wildcard_search(pattern)
        
        search_time = time.time() - start_time
        
        if not matching_terms:
            print(f"   ✗ No matching terms found")
            print(f"   ⏱️  Search time: {search_time:.3f}s")
            return []
        
        print(f"   ✓ Found {len(matching_terms)} matching term(s)")
        print(f"   Terms: {', '.join(list(matching_terms)[:10])}")
        if len(matching_terms) > 10:
            print(f"   ... and {len(matching_terms) - 10} more")
        print(f"   ⏱️  Search time: {search_time:.3f}s")
        
        # Find documents containing these terms
        matching_docs = set()
        term_doc_counts = []
        
        for term in matching_terms:
            docs = self.term_docs.get(term, set())
            term_doc_counts.append((term, len(docs)))
            matching_docs.update(docs)
        
        print(f"   📄 Documents found: {len(matching_docs)}")
        
        # Get document details
        results = []
        for doc_id in matching_docs:
            doc_info = self.documents[doc_id]
            
            # Find which terms from the pattern appear in this document
            doc_terms = self.doc_terms[doc_id]
            matched_terms = matching_terms.intersection(doc_terms)
            
            # Calculate score based on term frequency and pattern match
            score = len(matched_terms) * 10
            
            # Bonus for exact pattern match if it's a single term
            if len(matching_terms) == 1 and pattern.replace('*', '').replace('?', ''):
                if pattern in matched_terms:
                    score += 20
            
            results.append({
                'doc_id': doc_id,
                'name': doc_info['name'],
                'token_count': doc_info['token_count'],
                'match_score': score,
                'matched_terms': list(matched_terms),
                'matched_count': len(matched_terms),
                'pattern': pattern
            })
        
        # Sort by score
        results.sort(key=lambda x: x['match_score'], reverse=True)
        
        return results
    
    def phrase_wildcard_search(self, phrase_pattern: str) -> List[Dict]:
        """
        Wildcard search for phrases (multiple words with wildcards)
        
        Args:
            phrase_pattern: Phrase with wildcards (e.g., "supreme c*", "* of *")
        
        Returns:
            List of matching documents
        """
        if not phrase_pattern or not phrase_pattern.strip():
            return []
        
        print(f"\n🔍 Phrase wildcard search: '{phrase_pattern}'")
        
        # Split phrase into words
        words = phrase_pattern.lower().strip().split()
        
        if len(words) == 1:
            # Single word, use regular wildcard search
            return self.wildcard_search(words[0])
        
        # For each word in phrase, find matching terms
        word_term_sets = []
        
        for word in words:
            if '*' in word or '?' in word:
                matching_terms = self.wildcard_search(word, use_kgram=True)
                word_term_sets.append(matching_terms)
            else:
                # Exact word
                if word in self.term_docs:
                    word_term_sets.append({word})
                else:
                    word_term_sets.append(set())
        
        # Check if any word has no matches
        for i, term_set in enumerate(word_term_sets):
            if not term_set:
                print(f"   ✗ No matches for word: '{words[i]}'")
                return []
        
        print(f"   ✓ All words have matching terms")
        
        # Find documents containing sequences of these terms
        # This is simplified - in reality would need position information
        all_docs = set(self.documents.keys())
        
        # Find documents that contain at least one matching term for each word position
        candidate_docs = all_docs.copy()
        
        for term_set in word_term_sets:
            docs_with_any_term = set()
            for term in term_set:
                docs_with_any_term.update(self.term_docs.get(term, set()))
            
            candidate_docs = candidate_docs.intersection(docs_with_any_term)
            
            if not candidate_docs:
                print("   ✗ No documents contain all required terms")
                return []
        
        print(f"   📄 Candidate documents: {len(candidate_docs)}")
        
        # Score documents
        results = []
        for doc_id in candidate_docs:
            doc_info = self.documents[doc_id]
            doc_terms = self.doc_terms[doc_id]
            
            # Count how many words from phrase have matches in document
            matched_words = 0
            matched_terms_list = []
            
            for i, term_set in enumerate(word_term_sets):
                # Check if any term from this word's set is in document
                if term_set.intersection(doc_terms):
                    matched_words += 1
                    matched_terms = term_set.intersection(doc_terms)
                    matched_terms_list.append(list(matched_terms)[0])
            
            score = (matched_words / len(words)) * 100
            
            results.append({
                'doc_id': doc_id,
                'name': doc_info['name'],
                'token_count': doc_info['token_count'],
                'match_score': score,
                'matched_terms': matched_terms_list,
                'matched_words': matched_words,
                'total_words': len(words),
                'pattern': phrase_pattern
            })
        
        results.sort(key=lambda x: x['match_score'], reverse=True)
        return results
    
    def prefix_search(self, prefix: str) -> List[Dict]:
        """
        Prefix search (special case of wildcard: term*)
        
        Args:
            prefix: Prefix to search for
        
        Returns:
            List of matching documents
        """
        pattern = f"{prefix}*"
        return self.wildcard_search(pattern)
    
    def suffix_search(self, suffix: str) -> List[Dict]:
        """
        Suffix search (special case of wildcard: *term)
        
        Args:
            suffix: Suffix to search for
        
        Returns:
            List of matching documents
        """
        pattern = f"*{suffix}"
        return self.wildcard_search(pattern)
    
    def contains_search(self, substring: str) -> List[Dict]:
        """
        Contains search (special case of wildcard: *term*)
        
        Args:
            substring: Substring to search for
        
        Returns:
            List of matching documents
        """
        pattern = f"*{substring}*"
        return self.wildcard_search(pattern)
    
    def show_term_statistics(self, pattern: str = None):
        """Show statistics about terms matching a pattern"""
        if pattern:
            matching_terms = self.regex_wildcard_search(pattern)
            if not matching_terms:
                print(f"❌ No terms match pattern: '{pattern}'")
                return
            
            print(f"\n📊 TERM STATISTICS for pattern: '{pattern}'")
            print("=" * 80)
            print(f"   Matching terms: {len(matching_terms)}")
            
            # Show term frequencies
            term_freqs = []
            for term in matching_terms:
                doc_count = len(self.term_docs.get(term, set()))
                term_freqs.append((term, doc_count))
            
            # Sort by frequency
            term_freqs.sort(key=lambda x: x[1], reverse=True)
            
            print(f"\n📈 TOP 20 MATCHING TERMS BY DOCUMENT FREQUENCY:")
            print("-" * 80)
            for i, (term, freq) in enumerate(term_freqs[:20], 1):
                percentage = (freq / self.stats['total_documents']) * 100
                print(f"{i:3d}. {term:<25s} {freq:5d} docs ({percentage:.1f}%)")
        
        else:
            # Show general statistics
            print(f"\n📊 TERM STATISTICS")
            print("=" * 80)
            print(f"   Total unique terms: {self.stats['unique_terms']:,}")
            print(f"   Total term occurrences: {self.stats['total_terms']:,}")
            
            # Show most common terms
            term_freqs = []
            for term, docs in self.term_docs.items():
                term_freqs.append((term, len(docs)))
            
            term_freqs.sort(key=lambda x: x[1], reverse=True)
            
            print(f"\n📈 TOP 20 MOST COMMON TERMS:")
            print("-" * 80)
            for i, (term, freq) in enumerate(term_freqs[:20], 1):
                percentage = (freq / self.stats['total_documents']) * 100
                print(f"{i:3d}. {term:<25s} {freq:5d} docs ({percentage:.1f}%)")
        
        print("=" * 80)
    
    def find_similar_terms(self, term: str, max_distance: int = 2) -> List[str]:
        """
        Find terms similar to given term using wildcard patterns
        
        Args:
            term: Base term
            max_distance: Maximum edit distance
        
        Returns:
            List of similar terms
        """
        if term not in self.term_docs:
            print(f"❌ Term not found: '{term}'")
            return []
        
        similar_terms = set()
        
        # Generate wildcard patterns for similar terms
        patterns = []
        
        # 1. Prefix variations
        for i in range(1, min(max_distance + 1, len(term))):
            pattern = term[:i] + "*"
            patterns.append(pattern)
        
        # 2. Suffix variations
        for i in range(1, min(max_distance + 1, len(term))):
            pattern = "*" + term[-i:]
            patterns.append(pattern)
        
        # 3. Contains variations
        if len(term) > 3:
            for i in range(1, len(term) - 1):
                for j in range(i + 1, len(term)):
                    substring = term[i:j]
                    if len(substring) >= 2:
                        pattern = f"*{substring}*"
                        patterns.append(pattern)
        
        # Search for terms matching these patterns
        for pattern in set(patterns):  # Remove duplicates
            matching_terms = self.regex_wildcard_search(pattern)
            similar_terms.update(matching_terms)
        
        # Remove the original term
        similar_terms.discard(term)
        
        if similar_terms:
            print(f"\n🔍 Terms similar to '{term}':")
            sorted_terms = sorted(similar_terms)
            for i, similar_term in enumerate(sorted_terms[:20], 1):
                doc_count = len(self.term_docs.get(similar_term, set()))
                print(f"  {i:2d}. {similar_term} ({doc_count} docs)")
            
            if len(sorted_terms) > 20:
                print(f"  ... and {len(sorted_terms) - 20} more")
        
        return list(similar_terms)
    
    def display_results(self, results: List[Dict], title: str):
        """Display search results"""
        if not results:
            print(f"\n❌ No results found")
            return
        
        print(f"\n✅ {title}")
        print(f"📊 Found {len(results)} document(s)")
        print("=" * 80)
        
        for i, result in enumerate(results[:15], 1):
            print(f"\n{i:2d}. 📄 {result['name']}")
            print(f"    📏 Length: {result['token_count']:,} tokens")
            print(f"    ⭐ Score: {result['match_score']:.1f}")
            
            if 'matched_terms' in result and result['matched_terms']:
                terms_str = ', '.join(result['matched_terms'][:5])
                print(f"    🔍 Matched terms: {terms_str}")
                if len(result['matched_terms']) > 5:
                    print(f"    ... and {len(result['matched_terms']) - 5} more")
            
            if 'matched_count' in result:
                print(f"    📈 Terms matched: {result['matched_count']}")
        
        if len(results) > 15:
            print(f"\n... and {len(results) - 15} more documents")
        
        print("=" * 80)
        
        # Ask for document preview
        if results:
            choice = input("\n📖 Preview a document? (enter number or 'n'): ").strip()
            if choice.lower() != 'n' and choice.isdigit():
                idx = int(choice) - 1
                if 0 <= idx < len(results):
                    self.show_document_preview(results[idx]['doc_id'])
    
    def show_document_preview(self, doc_id: str, context_lines: int = 10):
        """Show document preview with matching terms highlighted"""
        if doc_id not in self.doc_terms:
            print(f"❌ Document not found: {doc_id}")
            return
        
        doc_info = self.documents[doc_id]
        
        # Try to load the actual document text
        doc_path = os.path.join(self.corpus_folder, "cleaned_docs", doc_info['name'])
        
        if not os.path.exists(doc_path):
            # Try alternative location
            doc_path = os.path.join(self.corpus_folder, doc_info['name'])
        
        if os.path.exists(doc_path):
            try:
                with open(doc_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                
                print(f"\n" + "=" * 80)
                print(f"📄 DOCUMENT: {doc_info['name']}")
                print(f"📏 Tokens: {doc_info['token_count']:,}")
                print("=" * 80)
                
                # Show first N lines
                lines = content.split('\n')[:context_lines]
                for i, line in enumerate(lines, 1):
                    if line.strip():
                        # Truncate long lines
                        if len(line) > 120:
                            line = line[:117] + "..."
                        print(f"{i:3d}. {line}")
                
                if len(content.split('\n')) > context_lines:
                    print(f"\n... and {len(content.split('\n')) - context_lines} more lines")
                
            except Exception as e:
                print(f"❌ Error reading document: {e}")
                print(f"\n📄 Document: {doc_info['name']}")
                print(f"📏 Tokens: {doc_info['token_count']:,}")
                print(f"🔍 Terms in document ({len(self.doc_terms[doc_id])}):")
                terms = sorted(self.doc_terms[doc_id])
                for i in range(0, len(terms), 10):
                    print(f"  {', '.join(terms[i:i+10])}")
        else:
            print(f"\n📄 Document: {doc_info['name']} (file not found)")
            print(f"📏 Tokens: {doc_info['token_count']:,}")
            print(f"🔍 Terms in document ({len(self.doc_terms[doc_id])}):")
            terms = sorted(self.doc_terms[doc_id])
            for i in range(0, len(terms), 10):
                print(f"  {', '.join(terms[i:i+10])}")
        
        print("=" * 80)
    
    def interactive_search(self):
        """Interactive wildcard search interface"""
        print("\n" + "=" * 80)
        print("🔍 WILDCARD SEARCH SYSTEM")
        print("=" * 80)
        print("\n📋 WILDCARD SYNTAX:")
        print("  • *        = Matches any sequence of characters")
        print("  • ?        = Matches any single character")
        print("  • term*    = Terms starting with 'term'")
        print("  • *term    = Terms ending with 'term'")
        print("  • *term*   = Terms containing 'term'")
        print("  • c?urt    = Terms like 'court', 'curt', etc.")
        print("  • m*rder   = Terms like 'murder', 'morder', etc.")
        
        print("\n📋 AVAILABLE COMMANDS:")
        print("  • search <pattern>    - Wildcard search")
        print("  • prefix <text>       - Prefix search (text*)")
        print("  • suffix <text>       - Suffix search (*text)")
        print("  • contains <text>     - Contains search (*text*)")
        print("  • phrase <pattern>    - Phrase with wildcards")
        print("  • similar <term>      - Find similar terms")
        print("  • terms [pattern]     - Show term statistics")
        print("  • stats               - Show index statistics")
        print("  • example             - Show examples")
        print("  • quit                - Exit")
        print("=" * 80)
        
        while True:
            user_input = input("\n🎯 Enter command: ").strip()
            
            if not user_input:
                continue
            
            if user_input.lower() == 'quit':
                print("👋 Goodbye!")
                break
            
            elif user_input.lower() == 'stats':
                print(f"\n📊 WILDCARD INDEX STATISTICS")
                print("=" * 60)
                print(f"Documents: {self.stats['total_documents']:,}")
                print(f"Unique terms: {self.stats['unique_terms']:,}")
                print(f"K-gram size: {self.stats['kgram_size']}")
                print(f"Unique k-grams: {len(self.kgram_index):,}")
                print("=" * 60)
            
            elif user_input.lower() == 'example':
                self.show_examples()
            
            elif user_input.lower().startswith('search '):
                pattern = user_input[7:].strip()
                if pattern:
                    results = self.wildcard_search(pattern, use_kgram=True)
                    self.display_results(results, f"Wildcard search: '{pattern}'")
                else:
                    print("❌ Please enter a wildcard pattern")
            
            elif user_input.lower().startswith('prefix '):
                text = user_input[7:].strip()
                if text:
                    results = self.prefix_search(text)
                    self.display_results(results, f"Prefix search: '{text}*'")
                else:
                    print("❌ Please enter prefix text")
            
            elif user_input.lower().startswith('suffix '):
                text = user_input[7:].strip()
                if text:
                    results = self.suffix_search(text)
                    self.display_results(results, f"Suffix search: '*{text}'")
                else:
                    print("❌ Please enter suffix text")
            
            elif user_input.lower().startswith('contains '):
                text = user_input[9:].strip()
                if text:
                    results = self.contains_search(text)
                    self.display_results(results, f"Contains search: '*{text}*'")
                else:
                    print("❌ Please enter text to search for")
            
            elif user_input.lower().startswith('phrase '):
                pattern = user_input[7:].strip()
                if pattern:
                    results = self.phrase_wildcard_search(pattern)
                    self.display_results(results, f"Phrase wildcard: '{pattern}'")
                else:
                    print("❌ Please enter a phrase pattern")
            
            elif user_input.lower().startswith('similar '):
                term = user_input[8:].strip()
                if term:
                    self.find_similar_terms(term)
                else:
                    print("❌ Please enter a term")
            
            elif user_input.lower().startswith('terms '):
                pattern = user_input[6:].strip()
                self.show_term_statistics(pattern)
            
            elif user_input.lower() == 'terms':
                self.show_term_statistics()
            
            else:
                # Default to wildcard search
                results = self.wildcard_search(user_input, use_kgram=True)
                self.display_results(results, f"Search: '{user_input}'")
    
    def show_examples(self):
        """Show example wildcard searches"""
        print("\n📝 WILDCARD SEARCH EXAMPLES")
        print("=" * 80)
        
        examples = [
            ("Prefix search", "court*", "Finds: court, courts, courtroom, etc."),
            ("Suffix search", "*ment", "Finds: judgment, document, agreement, etc."),
            ("Contains search", "*evid*", "Finds: evidence, evident, evidentiary, etc."),
            ("Single character wildcard", "c?urt", "Finds: court, curt"),
            ("Multiple wildcards", "m*rder", "Finds: murder, morder, murderer"),
            ("Combined wildcards", "*cour*ment*", "Finds terms containing both 'cour' and 'ment'"),
            ("Phrase with wildcards", "supreme c* of *", "Finds phrases like 'supreme court of pakistan'"),
        ]
        
        for category, pattern, explanation in examples:
            print(f"\n{category}:")
            print(f"  Pattern: {pattern}")
            print(f"  {explanation}")
        
        print("\n💡 LEGAL TERM EXAMPLES:")
        legal_examples = [
            ("judg*", "judgment, judge, judging, judgements"),
            ("*appeal*", "appeal, appeals, appealing, appealable"),
            ("evid*", "evidence, evident, evidentiary"),
            ("crimin*", "criminal, crime, criminology"),
            ("const*", "constitutional, constitution, constituted"),
            ("*right*", "right, rights, rightful, birthright"),
        ]
        
        for pattern, finds in legal_examples:
            print(f"  {pattern:<15} → {finds}")
        
        print("=" * 80)

import time  # Added import for timing

def main():
    """Main function"""
    print("=" * 80)
    print("🔍 WILDCARD QUERY SEARCH SYSTEM")
    print("=" * 80)
    
    # Try to find corpus folder
    corpus_folder = r"C:\Users\Armaghan Rafique\Desktop\AI Project\cleaned_corpus"
    
    if not os.path.exists(corpus_folder):
        print(f"❌ Corpus folder not found: {corpus_folder}")
        
        # Try alternative
        alt_folders = [
            r"C:\Users\Armaghan Rafique\Desktop\AI Project\supreme_court_judgements_txt",
            r"C:\Users\Armaghan Rafique\Desktop\AI Project",
            os.path.join(os.path.expanduser("~"), "Desktop", "AI Project", "cleaned_corpus")
        ]
        
        found = False
        for folder in alt_folders:
            if os.path.exists(folder):
                corpus_folder = folder
                print(f"✅ Using folder: {corpus_folder}")
                found = True
                break
        
        if not found:
            corpus_folder = input("📁 Enter corpus folder path: ").strip()
            if not os.path.exists(corpus_folder):
                print(f"❌ Folder does not exist: {corpus_folder}")
                return
    
    print(f"\n📁 Corpus folder: {corpus_folder}")
    
    # Create wildcard search system
    wildcard_system = WildcardSearchSystem(corpus_folder)
    
    # Load or build index
    if not wildcard_system.load_index():
        print("❌ Failed to initialize wildcard search system")
        return
    
    # Show welcome message
    print(f"\n🎯 WILDCARD SEARCH READY")
    print(f"   Documents: {wildcard_system.stats['total_documents']:,}")
    print(f"   Unique terms: {wildcard_system.stats['unique_terms']:,}")
    
    # Test some example searches
    print("\n" + "=" * 80)
    print("🧪 QUICK TEST EXAMPLES")
    print("=" * 80)
    
    test_patterns = [
        "court*",
        "*evidence",
        "*murder*",
        "judg*",
    ]
    
    for pattern in test_patterns[:2]:  # Test first 2
        print(f"\n🔍 Testing: '{pattern}'")
        results = wildcard_system.wildcard_search(pattern, use_kgram=True)
        if results:
            print(f"   ✅ Found {len(results)} documents")
            print(f"   Sample terms: {results[0].get('matched_terms', [])[:3]}")
        else:
            print(f"   ⚠️  No matches found")
    
    # Start interactive search
    wildcard_system.interactive_search()

if __name__ == "__main__":
    main()

🔍 WILDCARD QUERY SEARCH SYSTEM

📁 Corpus folder: C:\Users\Armaghan Rafique\Desktop\AI Project\cleaned_corpus
📂 Loading index from: C:\Users\Armaghan Rafique\Desktop\AI Project\cleaned_corpus\wildcard_index\wildcard_index.pkl
✅ Wildcard index loaded successfully!
   Documents: 1,460
   Unique terms: 15,240
   K-gram size: 3

🎯 WILDCARD SEARCH READY
   Documents: 1,460
   Unique terms: 15,240

🧪 QUICK TEST EXAMPLES

🔍 Testing: 'court*'

🔍 Wildcard search: 'court*'
   Method: k-gram indexing
   ✓ Found 1 matching term(s)
   Terms: court
   ⏱️  Search time: 0.000s
   📄 Documents found: 346
   ✅ Found 346 documents
   Sample terms: ['court']

🔍 Testing: '*evidence'

🔍 Wildcard search: '*evidence'
   Method: k-gram indexing
   ✓ Found 1 matching term(s)
   Terms: evidence
   ⏱️  Search time: 0.000s
   📄 Documents found: 19
   ✅ Found 19 documents
   Sample terms: ['evidence']

🔍 WILDCARD SEARCH SYSTEM

📋 WILDCARD SYNTAX:
  • *        = Matches any sequence of characters
  • ?        = Matche


🎯 Enter command:  search murd



🔍 Wildcard search: 'murd'
   Method: k-gram indexing
   ✗ No matching terms found
   ⏱️  Search time: 0.000s

❌ No results found



🎯 Enter command:  search



🔍 Wildcard search: 'search'
   Method: k-gram indexing
   ✓ Found 1 matching term(s)
   Terms: search
   ⏱️  Search time: 0.000s
   📄 Documents found: 32

✅ Search: 'search'
📊 Found 32 document(s)

 1. 📄 Cr_PLA_No_022015_Ashrat.txt
    📏 Length: 413 tokens
    ⭐ Score: 30.0
    🔍 Matched terms: search
    📈 Terms matched: 1

 2. 📄 Cr_Appeal_No_032011_in_CrPLA_No_102010_Naveed_Hussain_versus_the_State.txt
    📏 Length: 1,650 tokens
    ⭐ Score: 30.0
    🔍 Matched terms: search
    📈 Terms matched: 1

 3. 📄 Provincial_Government_Vs_Mst_Masnona_Shezadi_Provincial_20Govt_20Vr_20Mst._20Masnona.txt
    📏 Length: 442 tokens
    ⭐ Score: 30.0
    🔍 Matched terms: search
    📈 Terms matched: 1

 4. 📄 Cr_Misc_No072010_assistant_collector_custom_vs_syed_m._riaz.txt
    📏 Length: 793 tokens
    ⭐ Score: 30.0
    🔍 Matched terms: search
    📈 Terms matched: 1

 5. 📄 Cr_Appeal_No_022015_in_CrPLA_No_062013_Sher_20Wali_20versus_20Sakhawat_20Dar1.txt
    📏 Length: 897 tokens
    ⭐ Score: 30.0
    🔍 Ma

...


🔍 Wildcard search: 'court*'
   Method: k-gram indexing
   ✓ Found 1 matching term(s)
   Terms: court
   ⏱️  Search time: 0.000s
   📄 Documents found: 346

✅ Search: 'court*'
📊 Found 346 document(s)

 1. 📄 CPLA_Under_Objection_No1452019_Provincial_Govt_17._20judgment_20of_20mehboob_20hussain.txt
    📏 Length: 948 tokens
    ⭐ Score: 10.0
    🔍 Matched terms: court
    📈 Terms matched: 1

 2. 📄 Mst_Roqia_Begum_and_others_versus_Safdar_Ali_and_Mst._20Roqia_20Begum_20versus_20Safdar_20Ali.txt
    📏 Length: 1,037 tokens
    ⭐ Score: 10.0
    🔍 Matched terms: court
    📈 Terms matched: 1

 3. 📄 Cr_Appeal_No_082017_in_Cr_PLA_No_312015_the_20State_20versus_20Aziz-ur-Rehman_20bail_20granted.txt
    📏 Length: 430 tokens
    ⭐ Score: 10.0
    🔍 Matched terms: court
    📈 Terms matched: 1

 4. 📄 Civil_Review_No_122017_in_Civil_Appeal_No_6620_Abdul_20Murad_20Khan_20vs_20RO_20of_20Gojalti_20Review.txt
    📏 Length: 73 tokens
    ⭐ Score: 10.0
    🔍 Matched terms: court
    📈 Terms matched: 1

 5. 📄


📖 Preview a document? (enter number or 'n'):  17



📄 DOCUMENT: Cr_PLA_No_092017_The_20State_20versus_20Ghafoor_20__20others.txt
📏 Tokens: 254
  1. page in the supreme appellate court gilgit-baltistan, gilgit. chief judge. judge. cr. pla. no. the state through depu...



🎯 Enter command:  quit


👋 Goodbye!
